# 🧠 Electriz UCI Tests - Vers Tournoi Ready

**Objectif** : Valider python-chess pour UCI engine.

- Visez **>20M NPS**
- AlphaBeta d4 **<1s**
- PGN parse OK

In [2]:
import chess
import time
import sys
print(f'✅ python-chess v{chess.__version__}')

board = chess.Board()
print('\n📋 Position départ:')
print(board)

✅ python-chess v1.11.2

📋 Position départ:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [ ]:
moves = list(board.legal_moves)
print(f'🎯 {len(moves)} moves')
print('Premiers UCI:', [str(m) for m in moves])

board.push_uci('e2e4')
print(f'\n📍 FEN: {board.fen()}')
print(f'\n♟️ Après e4: \n{board}')

board.pop()

🎯 20 moves
Premiers UCI: ['g1h3', 'g1f3', 'b1c3', 'b1a3', 'h2h3', 'g2g3', 'f2f3', 'e2e3', 'd2d3', 'c2c3', 'b2b3', 'a2a3', 'h2h4', 'g2g4', 'f2f4', 'e2e4', 'd2d4', 'c2c4', 'b2b4', 'a2a4']

♟️ Après e4: 
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

📍 FEN: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1


Move.from_uci('e2e4')

In [ ]:
def parse_uci_position(cmd_str):
    tokens = cmd_str.split()
    board = chess.Board()
    i = 1
    if tokens[i] == 'fen':
        fen_end = tokens.index('moves') if 'moves' in tokens else len(tokens)
        fen = ' '.join(tokens[i+1:fen_end])
        board.set_fen(fen)
        i = fen_end
    
    while i < len(tokens):
        if tokens[i] == 'moves':
            for uci_move in tokens[i+1:]:
                board.push_uci(uci_move)
            break
        i += 1
    return board

# Tests
test_cmds = [
    'position startpos',
    'position startpos moves e2e4 e7e5',
    'position fen rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1 moves e7e5'
]

for cmd in test_cmds:
    b = parse_uci_position(cmd)
    print(f'✅ {cmd[:40]}... → {len(list(b.legal_moves))} moves')

In [ ]:
PIECE_VALUES = {chess.PAWN: 82, chess.KNIGHT: 337, chess.BISHOP: 365, chess.ROOK: 477, chess.QUEEN: 1025, chess.KING: 0}

def material_eval(board):
    score = sum(PIECE_VALUES.get(p.piece_type, 0) * (1 if p.color == chess.WHITE else -1) for p in board.piece_map().values())
    return score if board.turn == chess.WHITE else -score

print('⚖️ Eval départ:', material_eval(chess.Board()))

# BENCHMARK 🚀
board = chess.Board()
start = time.time()
nodes = 0
for _ in range(10**6):
    material_eval(board)
    nodes += 1
    if board.legal_moves:
        m = list(board.legal_moves)[0]
        board.push(m)
        board.pop()

nps = nodes / (time.time() - start)
print(f'🟢 {nps:,.0f} NPS' if nps > 20e6 else f'🔴 {nps:,.0f} NPS - Optimiser!')

In [ ]:
# PGN Parser
def extract_game_positions(pgn_path, max_games=1):
    positions = []
    try:
        with open(pgn_path, 'r', encoding='utf-8') as f:
            for game_num in range(max_games):
                game = chess.pgn.read_game(f)
                if game is None: break
                
                board = game.board()
                print(f'Game {game_num+1}: {game.headers["White"]}')
                
                for i, move in enumerate(list(game.mainline_moves())[:10]):
                    positions.append({'fen': board.fen(), 'move': move.uci()})
                    board.push(move)
                    print(f'  {i+1}. {move.uci()}')
        return positions
    except FileNotFoundError:
        print('📁 PGN manquant - test dummy')
        return [{'fen': 'start', 'move': 'e2e4'}]

pgn_path = '../data/raw/Hikaru_all.pgn'
pos = extract_game_positions(pgn_path)
print(f'\n📊 {len(pos)} positions OK')

In [ ]:
def alphabeta(board, depth, alpha=-99999, beta=99999, max_player=True):
    if depth == 0 or board.is_game_over():
        return material_eval(board)
    
    best = -99999 if max_player else 99999
    for move in board.legal_moves:
        board.push(move)
        val = alphabeta(board, depth-1, alpha, beta, not max_player)
        board.pop()
        
        if max_player:
            best = max(best, val)
            alpha = max(alpha, best)
        else:
            best = min(best, val)
            beta = min(beta, best)
        if beta <= alpha: break
    return best

board = chess.Board()
for d in [3, 4]:
    start = time.time()
    score = alphabeta(board, d)
    t = time.time() - start
    print(f'Depth {d}: {score} en {t:.2f}s')